In [ ]:
import sys
import numpy as np
import pandas as pd
import os
import matplotlib as plt
sys.path.append('..')
print(sys.path)
from nlp_pipeline.feature_extraction import FeatureExtractor
from nlp_pipeline.preprocess_text import TextPreprocessor
from model.evaluate_model import ModelEvaluator
from model.train_baseline import BaselineTrainer
from interpretation.lime_interpreter import LimeTextInterpreter
from interpretation.shap_interpreter import ShapInterpreter
from nlp_pipeline.back_translator import BackTranslationAugmentor
from nlp_pipeline.embedding_oversample import EmbeddingOversampler

['d:\\aco\\research\\Asd-classification\\notebook', 'c:\\Users\\MufliDevs\\anaconda3\\python312.zip', 'c:\\Users\\MufliDevs\\anaconda3\\DLLs', 'c:\\Users\\MufliDevs\\anaconda3\\Lib', 'c:\\Users\\MufliDevs\\anaconda3', '', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\setuptools\\_vendor', '..', '..']


Load dataset + Augmented

In [ ]:
df = df = pd.read_csv('../data/feature/merged_final.csv', encoding='latin1')
columns = ['label', 'transcription', 'total_words', 'unique_words', 'num_sentences',
                   'stopwords', 'num_adjectives', 'num_nouns', 'num_verbs', 'num_adverbs',
                   'type_token_ratio', 'avg_words_per_sentence']
df = df[columns]
augmentor = BackTranslationAugmentor()
df_augmented = augmentor.augment_dataframe(df)
print(df_augmented.head)
df_augmented.to_csv('../data/feature/1_augmented.csv', index=False)

Preprocessing text

In [3]:
df = df = pd.read_csv('../data/feature/combined_augmented_asd.csv', encoding='latin1')
text_preprocessor = TextPreprocessor()
df['clean_text'] = df['transcription'].apply(text_preprocessor.preprocess)
df[['transcription', 'clean_text']].head()

,transcription,clean_text
0,Tunjuk kaka coba,tunjuk kaka coba
1,Inii!,ini
2,Siapa namamu?,siapa nama
3,Iyaaaa?,
4,Kenalan duluu!,kenal duluu


In [ ]:
df['label'].value_counts()
df['label'].value_counts()

Ekstraksi fitur with TFIDF

In [4]:
extractor = FeatureExtractor()
X = extractor.fit_transform_tfidf(df['clean_text'])
y = df['label'].values
print(X.shape, y.shape)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

(8380, 5000) (8380,)


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Oversampling with TFIDF

In [5]:
X_dense = extractor.fit_transform_tfidf(df['clean_text']).toarray()

ling_cols = extractor.linguistic_cols
X_ling = df[ling_cols].values

y = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values

oversampler = EmbeddingOversampler()
X_embed_bal, X_ling_bal, y_bal = oversampler.oversample(X_dense, X_ling, y)


Ekstraksi fitur with IndoBERT

In [ ]:
extractor = FeatureExtractor()
X = extractor.encode_series_bert(df['clean_text'])
y = df['label'].values
print(X.shape, y.shape)

SVC WITH 5 FOLD

In [7]:
trainer = BaselineTrainer()

evaluator = ModelEvaluator(model_type='svm',pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)


[Oversampled] Fold 1
              precision    recall  f1-score   support

           0      0.937     0.810     0.869       930
           1      0.832     0.945     0.885       930

    accuracy                          0.877      1860
   macro avg      0.884     0.877     0.877      1860
weighted avg      0.884     0.877     0.877      1860

Confusion Matrix:
 [[753 177]
 [ 51 879]]
Confusion Matrix (Fold 1):
[[753 177]
 [ 51 879]]

[Oversampled] Fold 2
              precision    recall  f1-score   support

           0      0.946     0.812     0.874       930
           1      0.835     0.954     0.890       929

    accuracy                          0.883      1859
   macro avg      0.891     0.883     0.882      1859
weighted avg      0.891     0.883     0.882      1859

Confusion Matrix:
 [[755 175]
 [ 43 886]]
Confusion Matrix (Fold 2):
[[755 175]
 [ 43 886]]

[Oversampled] Fold 3
              precision    recall  f1-score   support

           0      0.923     0.845     0.8

USE THIS IF YOU NOT USING BERT

In [9]:
final_model = trainer.train(X_embed_bal, y_bal)

lime_interpreter = LimeTextInterpreter(
    model=final_model,
    vectorizer=extractor.get_tfidf_vectorizer(),
    class_names=['NON ASD', 'ASD']
)

asd_samples = df[df['label'] == 'ASD'].sample(n=12, random_state=42)
non_asd_samples = df[df['label'] == 'NON ASD'].sample(n=12, random_state=42)
lime_samples = pd.concat([asd_samples, non_asd_samples]).reset_index(drop=True)

lime_interpreter.save_lime_explanation_to_csv(lime_samples,"../reports/lime_results.csv")

Berhasil menyimpan!


In [ ]:
final_model = BaselineTrainer('svm').train(X,y)
interpreter = ShapInterpreter(model=final_model)
sampled_df = df.sample(n=100, random_state=42)
texts = sampled_df['clean_text'].tolist()
labels = sampled_df['label'].tolist()

shap_values = interpreter.explain(texts)
interpreter.save_shap_explanations_to_csv(
    shap_values=shap_values,
    texts=texts,
    output_path="../reports/shap_results_SVM.csv",
    true_labels=labels
)

LINEAR REGRETION WITH 5 FOLD

In [11]:
trainer = BaselineTrainer()
evaluator = ModelEvaluator(pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)


[Oversampled] Fold 1
              precision    recall  f1-score   support

           0      0.921     0.794     0.853       930
           1      0.819     0.932     0.872       930

    accuracy                          0.863      1860
   macro avg      0.870     0.863     0.862      1860
weighted avg      0.870     0.863     0.862      1860

Confusion Matrix:
 [[738 192]
 [ 63 867]]
Confusion Matrix (Fold 1):
[[738 192]
 [ 63 867]]

[Oversampled] Fold 2
              precision    recall  f1-score   support

           0      0.940     0.797     0.863       930
           1      0.824     0.949     0.882       929

    accuracy                          0.873      1859
   macro avg      0.882     0.873     0.872      1859
weighted avg      0.882     0.873     0.872      1859

Confusion Matrix:
 [[741 189]
 [ 47 882]]
Confusion Matrix (Fold 2):
[[741 189]
 [ 47 882]]

[Oversampled] Fold 3
              precision    recall  f1-score   support

           0      0.927     0.820     0.8

In [ ]:
final_model = trainer.train(X, y)

lime_interpreter = LimeTextInterpreter(
    model=final_model,
    vectorizer=extractor.get_tfidf_vectorizer(),
    class_names=['NON ASD', 'ASD']
)

asd_samples = df[df['label'] == 'ASD'].sample(n=12, random_state=42)
non_asd_samples = df[df['label'] == 'NON ASD'].sample(n=12, random_state=42)
lime_samples = pd.concat([asd_samples, non_asd_samples]).reset_index(drop=True)

lime_interpreter.save_lime_explanation_to_csv(lime_samples,"../reports/lime_results_LOGRES.csv")

SVM WITH 10 FOLD

In [ ]:
trainer = BaselineTrainer()
evaluator = ModelEvaluator(model_type='svm', n_splits= 10, random_state=32,pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)


[Oversampled] Fold 1
              precision    recall  f1-score   support

           0      0.958     0.832     0.891       465
           1      0.852     0.963     0.904       465

    accuracy                          0.898       930
   macro avg      0.905     0.898     0.897       930
weighted avg      0.905     0.898     0.897       930

Confusion Matrix:
 [[387  78]
 [ 17 448]]
Confusion Matrix (Fold 1):
[[387  78]
 [ 17 448]]

[Oversampled] Fold 2
              precision    recall  f1-score   support

           0      0.936     0.815     0.871       465
           1      0.836     0.944     0.887       465

    accuracy                          0.880       930
   macro avg      0.886     0.880     0.879       930
weighted avg      0.886     0.880     0.879       930

Confusion Matrix:
 [[379  86]
 [ 26 439]]
Confusion Matrix (Fold 2):
[[379  86]
 [ 26 439]]

[Oversampled] Fold 3
              precision    recall  f1-score   support

           0      0.943     0.860     0.9

In [ ]:
final_model = trainer.train(X, y)

lime_interpreter = LimeTextInterpreter(
    model=final_model,
    vectorizer=extractor.get_tfidf_vectorizer(),
    class_names=['NON ASD', 'ASD']
)

asd_samples = df[df['label'] == 'ASD'].sample(n=12, random_state=42)
non_asd_samples = df[df['label'] == 'NON ASD'].sample(n=12, random_state=42)
lime_samples = pd.concat([asd_samples, non_asd_samples]).reset_index(drop=True)

lime_interpreter.save_lime_explanation_to_csv(lime_samples,"../reports/lime_results_SVM10FOLD.csv")

In [ ]:
final_model = BaselineTrainer('svm').train(X,y)
interpreter = ShapInterpreter(model=final_model)
sampled_df = df.sample(n=100, random_state=42)
texts = sampled_df['clean_text'].tolist()
labels = sampled_df['label'].tolist()

shap_values = interpreter.explain(texts)
interpreter.save_shap_explanations_to_csv(
    shap_values=shap_values,
    texts=texts,
    output_path="../reports/shap_results_SVM_10_fold.csv",
    true_labels=labels
)

LINEAR REGRETION WITH 10 FOLD

In [ ]:
trainer = BaselineTrainer()
evaluator = ModelEvaluator(n_splits= 10, random_state=32)

results = evaluator.cross_validate(X,y)

In [ ]:
final_model = BaselineTrainer('svm').train(X,y)
interpreter = ShapInterpreter(model=final_model)
sampled_df = df.sample(n=100, random_state=42)
texts = sampled_df['clean_text'].tolist()
labels = sampled_df['label'].tolist()

shap_values = interpreter.explain(texts)
interpreter.save_shap_explanations_to_csv(
    shap_values=shap_values,
    texts=texts,
    output_path="../reports/shap_results_Logres_10_fold.csv",
    true_labels=labels
)


In [ ]:
interpreter.visualize(shap_values= shap_values)

FUSION

In [ ]:
extractor = FeatureExtractor()
X = extractor.extract_fused_features_bert(df)
y = df['label'].values

In [ ]:
evaluator = ModelEvaluator(model_type='svm', n_splits= 10, random_state=32)
results = evaluator.cross_validate_with_confusionmatrix(X, y)


Testing Model

In [ ]:
trainer = BaselineTrainer(model_type='svm')
model = trainer.train(X, y)

new_text = "Apakah kamu suddah makan?"
clean_text = text_preprocessor.preprocess(new_text)
X_new = extractor.encode_series_bert([clean_text])
predicted_label = model.predict(X_new)[0]
print(f"Predicted label: {predicted_label}")

In [ ]:
new_text = "Aku takut aku takut aku takut aku takut aku takut aku takut aku takut"
clean_text = text_preprocessor.preprocess(new_text)
X_new = extractor.encode_series_bert([clean_text])
predicted_label = model.predict(X_new)[0]
print(f"Predicted label: {predicted_label}")